<div>
<img src=https://www.institutedata.com/wp-content/uploads/2019/10/iod_h_tp_primary_c.svg width="300">
</div>

# Lab 3.2.1 
# *Querying the International Space Station*

## The OpenNotify API

The OpenNotify API exposes a few attributes of the International Space Station (ISS) via a simple, authentication-free interface. The simplicity of this API precludes any need for a dedicated Python library. However, as with many APIs, it accepts requests according to HTTP standards and returns responses in JSON format, so the Python libraries request and json will make managing the I/O simpler still.

In [51]:
import requests
import json
from datetime import datetime, date, time

This request fetches the latest position of the international space station:

In [52]:
response = requests.get("http://api.open-notify.org/iss-now.json")
#http://api.open-notify.org/iss-now.json

In [54]:
overheads = response.json()
print(overheads)
print(overheads['timestamp'])

{'timestamp': 1656756984, 'message': 'success', 'iss_position': {'latitude': '22.6186', 'longitude': '-12.7045'}}
1656756984


Print the status code of the response:

In [55]:
response.headers['Content-Type']

'application/json'

We can also request the current position of the ISS and the next few times at which it will be over a certain location. The latitude and longitude of Sydney are (-33.87, 151.21). Create a dict named `parameters` whose elements are key:value pairs named `lat` and `lon`, containing those figures:

In [56]:
parameters ={"lat":-33.87, "lon":151.21}

The API request we want starts like the last one, but the last part of the endpoint is "iss-pass.json", and it takes a second argument -- namely, the `parameters` value created above. Compose and execute the API request:

In [57]:
response = requests.get("http://api.open-notify.org/iss-pass.json", params = parameters)
response

<Response [200]>

Print the response header:

In [58]:
print(response.headers)

{'Server': 'nginx/1.10.3', 'Date': 'Sat, 02 Jul 2022 10:17:05 GMT', 'Content-Type': 'application/json', 'Content-Length': '454', 'Connection': 'keep-alive', 'Via': '1.1 vegur'}


Print the content of the response (the data that the server returned):

In [59]:
print(response.content)

b'{\n  "message": "success", \n  "request": {\n    "altitude": 100, \n    "datetime": 1656756909, \n    "latitude": -33.87, \n    "longitude": 151.21, \n    "passes": 5\n  }, \n  "response": [\n    {\n      "duration": 639, \n      "risetime": 1656759276\n    }, \n    {\n      "duration": 181, \n      "risetime": 1656807941\n    }, \n    {\n      "duration": 646, \n      "risetime": 1656813468\n    }, \n    {\n      "duration": 603, \n      "risetime": 1656819304\n    }\n  ]\n}\n'


Print just the "content-type" value from the header:

In [61]:
print(response.headers['content-type'])

application/json


Note that this is a Python byte string:

In [60]:
print(type(response.content))

<class 'bytes'>


JSON was designed to be easy for computers to read, not for people. The requests library can decode the JSON byte string:

In [62]:
overheads = response.json()
print(overheads)
overheads  # I only have one line of code

{'message': 'success', 'request': {'altitude': 100, 'datetime': 1656756909, 'latitude': -33.87, 'longitude': 151.21, 'passes': 5}, 'response': [{'duration': 639, 'risetime': 1656759276}, {'duration': 181, 'risetime': 1656807941}, {'duration': 646, 'risetime': 1656813468}, {'duration': 603, 'risetime': 1656819304}]}


{'message': 'success',
 'request': {'altitude': 100,
  'datetime': 1656756909,
  'latitude': -33.87,
  'longitude': 151.21,
  'passes': 5},
 'response': [{'duration': 639, 'risetime': 1656759276},
  {'duration': 181, 'risetime': 1656807941},
  {'duration': 646, 'risetime': 1656813468},
  {'duration': 603, 'risetime': 1656819304}]}

What kind of object did this give us?

In [80]:
print(type(overheads))

<class 'dict'>


Python dicts are easier to work with, but the data we want is still buried in that data structure, so we have to dig it out. First, extract the passes value to a separate dict:

In [81]:
passes = overheads['response']
passes

[{'duration': 639, 'risetime': 1656759276},
 {'duration': 181, 'risetime': 1656807941},
 {'duration': 646, 'risetime': 1656813468},
 {'duration': 603, 'risetime': 1656819304}]

In [66]:
passes[1]['risetime']

1656807941

In [67]:
list = []
for i in passes:
    list.append(i['risetime'])
list

[1656759276, 1656807941, 1656813468, 1656819304]

Now extract the risetime strings into an array called srisetimes:

In [68]:
srisetimes = [xpass['risetime'] for xpass in passes]
srisetimes

[1656759276, 1656807941, 1656813468, 1656819304]

It is Unix Timestamp. We wonvert these to an array of Python datetime values called risetimes:

```
datetime.fromtimestamp(str)
```

Convert these to an array of Python `datetime` values called `risetimes`:

In [71]:
risetimes = [datetime.fromtimestamp(xpass['risetime']) for xpass in passes]
risetimes

[datetime.datetime(2022, 7, 2, 20, 54, 36),
 datetime.datetime(2022, 7, 3, 10, 25, 41),
 datetime.datetime(2022, 7, 3, 11, 57, 48),
 datetime.datetime(2022, 7, 3, 13, 35, 4)]

Finally, print these in a format that people understand:

In [72]:
for risetime in risetimes:
    print(risetime.strftime('%d/%m/%y %I:%M'))

02/07/22 08:54
03/07/22 10:25
03/07/22 11:57
03/07/22 01:35


Here is an endpoint that tells us who is onboard:

In [73]:
response = requests.get("http://api.open-notify.org/astros.json")

Referring to the methods used above, extract the number of astronauts and their names:

In [74]:
astros = response.json()
astros

{'number': 10,
 'people': [{'name': 'Oleg Artemyev', 'craft': 'ISS'},
  {'name': 'Denis Matveev', 'craft': 'ISS'},
  {'name': 'Sergey Korsakov', 'craft': 'ISS'},
  {'name': 'Kjell Lindgren', 'craft': 'ISS'},
  {'name': 'Bob Hines', 'craft': 'ISS'},
  {'name': 'Samantha Cristoforetti', 'craft': 'ISS'},
  {'name': 'Jessica Watkins', 'craft': 'ISS'},
  {'name': 'Cai Xuzhe', 'craft': 'Tiangong'},
  {'name': 'Chen Dong', 'craft': 'Tiangong'},
  {'name': 'Liu Yang', 'craft': 'Tiangong'}],
 'message': 'success'}

In [79]:
print(astros['number'])
[print(astronaut['name']) for astronaut in astros['people']]

10
Oleg Artemyev
Denis Matveev
Sergey Korsakov
Kjell Lindgren
Bob Hines
Samantha Cristoforetti
Jessica Watkins
Cai Xuzhe
Chen Dong
Liu Yang


[None, None, None, None, None, None, None, None, None, None]

JSON was designed to be easy for computers to read, not for people. The `requests` library can decode the JSON byte string:

In [13]:
astros['people']

[{'name': 'Oleg Artemyev', 'craft': 'ISS'},
 {'name': 'Denis Matveev', 'craft': 'ISS'},
 {'name': 'Sergey Korsakov', 'craft': 'ISS'},
 {'name': 'Kjell Lindgren', 'craft': 'ISS'},
 {'name': 'Bob Hines', 'craft': 'ISS'},
 {'name': 'Samantha Cristoforetti', 'craft': 'ISS'},
 {'name': 'Jessica Watkins', 'craft': 'ISS'},
 {'name': 'Cai Xuzhe', 'craft': 'Tiangong'},
 {'name': 'Chen Dong', 'craft': 'Tiangong'},
 {'name': 'Liu Yang', 'craft': 'Tiangong'}]

Finally, print these in a format that people understand:



```
str.strftime('%d/%m/%y %I:%M')

27/03/19 10:59
27/03/19 12:35
27/03/19 02:11
28/03/19 03:42
28/03/19 05:13
```



In [14]:
astros['people'][0]

{'name': 'Oleg Artemyev', 'craft': 'ISS'}

In [16]:
for i in astros['people']:
    print(i['name'])

Oleg Artemyev
Denis Matveev
Sergey Korsakov
Kjell Lindgren
Bob Hines
Samantha Cristoforetti
Jessica Watkins
Cai Xuzhe
Chen Dong
Liu Yang


## HOMEWORK


1. Write a simple handler for the response status code (refer to lab resources slide for HTTP response codes). As this Jupyter Notebook is an interactive device, the handler does not need to manage subsequent code execution (i.e. by branching or aborting execution), although it should return something that could be used to do so if deployed in a Python program.

In [ ]:
#ANSWER:
def handleResponse(response, verbose = False):
    '''
    Returns Boolean Value, Status Code, 
    '''
  # if Status Code is 200 return false, and status code
  # Otherwise Return True and Status Code

In [82]:
def handleResponse(response, verbose = False):
    if response.status_code == 200:
        if verbose:
            print('Response = 200: OK')
        fail = False
    else:
        if verbose:
            print('Response = ' + str(response.status_code) + ': ' + '(TODO: output of lookup function here)')
        fail = True
    return(fail, response.status_code)

2. Test your response handler on some correct and incorrect API calls.

In [84]:
response = requests.get("http://api.open-notify.org/astros.json")
response

<Response [200]>

In [83]:
response = requests.get("http://api.open-notify.org/astros.json")
if handleResponse(response)[0]:
    print('API call failed. Resolve issue before continuing!')
    
response = requests.get("http://api.open-notify.org/iss-pass.json")
handleResponse(response, True)[0]

Response = 400: (TODO: output of lookup function here)


True

>

>

>



---



---



> > > > > > > > > © 2022 Institute of Data


---



---



